In [ ]:
%pip install langchain-core langchain-openai langchain-exa

In [5]:
import os

os.environ["OPENAI_API_KEY"] = ""
os.environ["EXA_API_KEY"] = ""

In [8]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["LANGCHAIN_PROJECT"] = ""

In [7]:
from langchain_exa import ExaSearchRetriever

retriever = ExaSearchRetriever(k=3, highlights=True)
documents = retriever.invoke("Meilleur point de vue à Tokyo")
len(documents)

3

In [8]:
documents[0].metadata

{'title': 'Louis Vuitton dévoile la nouvelle façade hypnotique de sa boutique de Tokyo',
 'url': 'https://www.vogue.fr/vogue-hommes/article/louis-vuitton-boutique-tokyo',
 'id': 'ZmMtrrQq9xry5b5Ce8lGXA',
 'score': 0.17380395531654358,
 'published_date': '2021-03-24',
 'author': 'Alexandre Marain',
 'highlights': ["La boutique tokyoïte de Louis Vuitton fait peau neuve Si l'extérieur à l'esthétique rafraîchissante a été imaginé par Jun Aoki, l'intérieur de la boutique, piqué d'influences mais aussi de couleurs très naturelles et souligné par un escalier proéminent en chêne sculpté et bordé de verre, a été conçu par l'architecte américain Peter Marino, que l'on connaît pour ses nombreuses collaborations avec artistes et maisons de mode dont  Dior , Calvin Klein ou encore  Yves Saint Laurent . À l'arrière du magasin, un second escalier est agrémenté d'une gigantesque fresque sublimée de l'œuvre de Kimiko Fujimura, Wave Blue Line. Sans oublier le mobilier tout en courbe signé Morten Stenbae

In [9]:
print(documents[1].metadata["highlights"])

['En\xa02002, le Journal of Natural Disaster Science publie sa description du désastre à venir. Les autorités et l’opérateur Tokyo Electric Power Company (Tepco) rejettent ses mises en garde jusqu’en\xa02011. Comment le formidable bloc industriel, scientifique et politique de l’un des pays les plus développés du monde s’est-il aveuglé devant la certitude annoncée d’un immense désastre\xa0? C’est une construction historique. Tragique.']


In [10]:
print(documents[1].metadata["url"])

https://www.lemonde.fr/idees/article/2021/03/11/fukushima-comment-le-japon-s-est-aveugle-devant-la-certitude-annoncee-d-un-desastre_6072680_3232.html


In [11]:
from langchain_exa import ExaSearchRetriever
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda


retriever = ExaSearchRetriever(k=3, highlights=True)

document_prompt = PromptTemplate.from_template("""
<source>
    <url>{url}</url>
    <highlights>{highlights}</highlights>
</source>
""")

document_chain = RunnableLambda(
    lambda document: {
        "highlights": document.metadata["highlights"], 
        "url": document.metadata["url"]
    }
) | document_prompt

retrieval_chain = retriever | document_chain.map() | (lambda docs: "\n".join([i.text for i in docs]))

print(retrieval_chain.invoke("Meilleur endroit pour visiter le Japon"))


<source>
    <url>https://www.vogue.fr/culture/agenda/story/adresses-special-japon-a-paris/4321</url>
    <highlights>['01 83 97 00 00    crédit photo @londoneater       @londoneater  S’offrir un soin kobido au spa du Four Seasons George V En plus d’avoir fait peau neuve, le spa du Four Seasons George V initie désormais ses visiteurs aux joies des traditions esthétiques nippones. Avec en tête de liste le Kobido, un soin ancestral dont raffolait la cour impériale du Japon. Le principe ? Un massage facial très dynamique qui réveille l’épiderme et les muscles du visage, effectué par des mains expertes, formées chez le Dr Shogo Mochizuki de Tokyo. Soit un protocole qui apporte éclat et tonicité et qui propulse, le temps d’une petite heure, au Pays du Soleil-levant.']</highlights>
</source>


<source>
    <url>https://www.vogue.fr/lifestyle/voyages/diaporama/les-adresses-de-lou-doillon-au-japon/41961</url>
    <highlights>["En comprenant plus de 200, le lieu est propice aux promenades zen 

In [18]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from operator import itemgetter

generation_prompt = ChatPromptTemplate.from_messages([
    ("system", "Vous êtes un assistant de recherche expert. Vous utilisez du contexte en format xml pour répondre aux questions des utilisateurs."),
    ("human", """
Veuillez répondre à la question suivante en vous basant sur le contexte fourni. Veuillez citer vos sources à la fin de votre réponse.:
     
Query: {query}
---
<context>
{context}
</context>
""")
])

llm = ChatOpenAI()
chain =( 
    RunnablePassthrough.assign(context=(itemgetter("query") |retrieval_chain))
)| generation_prompt | llm

In [20]:
chain.invoke({"query":"Meilleurs lieux touristiques en Corée du Sud"}).content

"Selon les sources fournies, voici quelques-uns des meilleurs lieux touristiques en Corée du Sud :\n\n1. Des sites incontournables aux vallées méconnues en passant par la Corée des randonnées, la péninsule sud-coréenne offre une palette unique de circuits aux couleurs locales où nature et culture cohabitent en parfaite harmonie.\n\n2. La Corée du Sud est également réputée pour ses villes dynamiques comme Séoul, Busan et Incheon, ses montagnes majestueuses comme le mont Seoraksan, et ses temples anciens tels que le temple Bulguksa.\n\n3. Les amateurs de culture pourront visiter des palais royaux comme Gyeongbokgung, des quartiers traditionnels comme Bukchon Hanok Village, ou encore des musées d'art contemporain comme le Leeum Samsung Museum of Art.\n\n4. Pour les amoureux de la nature, les parcs nationaux de Jirisan et de Jeju Island offrent des paysages époustouflants avec des cascades, des forêts luxuriantes et des formations rocheuses uniques.\n\nCes suggestions de lieux touristiques

In [21]:
chain.invoke({"query":"Qu'avais-je dis avant?"}).content

'D\'après le contexte fourni, vous aviez dit : "Dans Paris, Paris dégoûtant / seuls les rats sont contents / ils savent qu\'ici les végans pas idiots / les nourrissent qu\'avec du bio." Cela fait partie des paroles de la chanson polémique de Pierre Perret qui critiquait la saleté de la capitale et l\'inaction de la mairie. Vous pouvez trouver plus d\'informations sur ce sujet en suivant ce lien : https://www.francetvinfo.fr/culture/musique/paris-saccage-pierre-perret-critique-paris-et-sa-mairie-dans-sa-derniere-chanson_5725826.html.\n\nSources :\n1. https://www.francetvinfo.fr/culture/musique/paris-saccage-pierre-perret-critique-paris-et-sa-mairie-dans-sa-derniere-chanson_5725826.html'

In [25]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

chain_with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)